## Import Packages

In [1]:
import azureml

from azureml.core import Workspace, Environment
from azureml.core.authentication import AzureCliAuthentication
from azureml.core.conda_dependencies import CondaDependencies 

print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.41.0


## Connect to the Workspace

In [2]:
cli_auth = AzureCliAuthentication()

ws = Workspace(
    subscription_id="d752c461-4a37-4d63-a15d-ec58b07063cb",
    resource_group="ml-poc",
    workspace_name="ml-poc-ws",
    auth=cli_auth,
)

print("Found workspace {} at location {}".format(ws.name, ws.location))

Found workspace ml-poc-ws at location westeurope


## Create train environment

In [8]:
CUSTOM_TRAIN_GPU_IMAGE = 'mcr.microsoft.com/azureml/openmpi4.1.0-cuda11.1-cudnn8-ubuntu18.04:20220516.v1'

CUSTOM_TRAIN_GPU_DOCKERFILE = f'''
FROM {CUSTOM_TRAIN_GPU_IMAGE}

RUN apt-get install ffmpeg libsm6 libxext6 libgl1-mesa-glx  -y
RUN apt-get update && apt-get install -y python3-opencv
'''

In [9]:
conda_channels = ['anaconda', 'pytorch', 'conda-forge']

conda_packages=[
    'pip==20.2.4', 'pytorch=1.10.0', 'torchvision=0.11.1', 'torchaudio=0.10.0',
    'cudatoolkit=11.1.1', 'nvidia-apex=0.1.0', 'gxx_linux-64',
]

pip_packages = [
    'matplotlib>=3.3,<3.4', 'scikit-learn==1.0.2', 'psutil>=5.8,<5.9',
    'tqdm>=4.59,<4.63', 'pandas>=1.3,<1.4', 'scipy>=1.5,<1.8',
    'numpy>=1.10,<1.22', 'ipykernel~=6.0', 'azureml-core==1.41.0.post3',
    'azureml-defaults==1.41.0', 'azureml-mlflow==1.41.0', 'azureml-telemetry==1.41.0',
    'tensorboard==2.6.0', 'tensorflow-gpu==2.6.0', 'onnxruntime-gpu>=1.7,<1.10',
    'future==0.18.2', 'opencv-python==4.5.4.60', 'accelerate==0.9.0',
    'opencv-python-headless==4.5.5.64', 'albumentations==1.1.0', 'torch-tb-profiler==0.3.1',
    'protobuf==3.20.0',
]

In [10]:
conda_deps = CondaDependencies.create(
    python_version="3.8", conda_packages=conda_packages, pip_packages=pip_packages,
)

for c in conda_channels:
    conda_deps.add_channel(c)

train_environment = Environment(name="flowers-train-pytorch-1-10-ubuntu18-py38-cuda11-gpu")
train_environment.docker.base_image = None
train_environment.docker.base_dockerfile = CUSTOM_TRAIN_GPU_DOCKERFILE
train_environment.python.conda_dependencies = conda_deps
# train_environment.python.interpreter_path = 'python'
# train_environment.python.user_managed_dependencies=True
# train_environment.python.interpreter_path = "/opt/miniconda/bin/python"
train_environment.register(ws);

In [19]:
# details = train_environment.get_image_details(workspace=ws)
# print(details['ingredients']['dockerfile'])

## Create infer environment

In [3]:
CUSTOM_INFER_GPU_IMAGE = 'mcr.microsoft.com/azureml/openmpi4.1.0-cuda11.1-cudnn8-ubuntu18.04:20220516.v1'

CUSTOM_INFER_GPU_DOCKERFILE = f'''
FROM {CUSTOM_INFER_GPU_IMAGE}

RUN apt-get install ffmpeg libsm6 libxext6 libgl1-mesa-glx  -y
RUN apt-get update && apt-get install -y python3-opencv
'''

In [4]:
conda_channels = ['anaconda', 'pytorch', 'conda-forge']

conda_packages=[
    'pip==20.2.4', 'pytorch=1.10.0', 'torchvision=0.11.1',
    'cudatoolkit=11.1.1', 'nvidia-apex=0.1.0', 'gxx_linux-64',
]

pip_packages = [
    'psutil>=5.8,<5.9', 'tqdm>=4.59,<4.63', 'numpy>=1.10,<1.22', 'ipykernel~=6.0',
    'azureml-core==1.41.0.post3', 'azureml-defaults==1.41.0', 'azureml-mlflow==1.41.0',
    'azureml-telemetry==1.41.0', 'onnxruntime-gpu>=1.7,<1.10', 'future==0.18.2',
    'opencv-python==4.5.4.60', 'opencv-python-headless==4.5.5.64', 'albumentations==1.1.0',
    'protobuf==3.20.0',
]

In [5]:
conda_deps = CondaDependencies.create(
    python_version="3.8", conda_packages=conda_packages, pip_packages=pip_packages,
)

for c in conda_channels:
    conda_deps.add_channel(c)

infer_environment = Environment(name="flowers-infer-pytorch-1-10-ubuntu18-py38-cuda11-gpu")
infer_environment.docker.base_image = None
infer_environment.docker.base_dockerfile = CUSTOM_INFER_GPU_DOCKERFILE
infer_environment.python.conda_dependencies = conda_deps
infer_environment.register(ws);